In [1]:
import os
os.getcwd()

'/data/zhaozibo/text-kgs-table/components/ft_data_prep'

In [3]:
import sys, os
sys.path.append('/data/zhaozibo/text-kgs-table')
import pandas as pd
from components.kgs.dataset_kgs.e2e_kg import *
from utils import visualize_knowledge_graph_interactive, KnowledgeGraph, visualize_knowledge_graph
from components.retriever.DocData import *

In [24]:
part = "train"
file_path = f'../../data/e2e/{part}.text'

# Reading the file line by line and storing each line in a list
lines = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        lines.append(line.strip())
lines[0:5]

['The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.',
 'Close to Café Brazil, The Cambridge Blue pub serves delicious Tuscan Beef for the cheap price of £10.50. Delicious Pub food.',
 'The Eagle is a low rated coffee shop near Burger King and the riverside that is family friendly and is less than £20 for Japanese food.',
 'Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.',
 'For luxurious French food, the Loch Fyne is located by the river next to The Rice Boat.']

In [6]:
from components.kgs.dataset_kgs.e2e_rotowire_field import fields_list_rotowire, fields_list_e2e
from components.kgs.dataset_kgs.CPL_field import fields_list_CPL
import json
import numpy as np
import pandas as pd

In [7]:
fields_list_e2e

{'Table1': {'Row_Name': {'Restaurant': 1},
  'Fields': {'Name': str,
   'Price range': ['More than £30',
    'Cheap',
    'Less than £20',
    '£20-25',
    'Moderate',
    'High'],
   'Customer rating': ['5 out of 5',
    'Low',
    'Average',
    '3 out of 5',
    '1 out of 5',
    'High'],
   'Near': str,
   'Food': ['Japanese',
    'French',
    'English',
    'Fast food',
    'Italian',
    'Indian',
    'Chinese'],
   'Area': ['Riverside', 'City centre'],
   'Family friendly': ['Yes', 'No']}}}

In [20]:
ie_prompt_extract_role_template="""
You are a useful assistant who can classify given proper nouns based on the list of role options and related information provided by the user.
1. Determine whether this proper noun belongs to any of the given role lists. If it does not belong to any of them, respond 'Not Role'.
2. If it belongs to a certain role in the role list, you only need to respond the options in the list. You cannot output any content not in the role list.
3. Respond to the user's question like the examples provided below:

Proper noun: Hawks
Related Context: The Atlanta Hawks (46 - 12) beat the Orlando Magic (19 - 41) 95 - 88 on Friday. Al Horford had a good all - around game, putting up 17 points, 13 rebounds.
Role option scope: ['Team']
Question: Dose the proper noun is a Role in the scope? If dose, which is?
Answer: Team

Proper noun: Vaults
Related Context: The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.
Role option scope: ['Restaurant']
Question: Dose the proper noun is a Role in the scope? If dose, which is?
Answer: Restaurant

Proper noun: Blue
Related Context: Close to Café Brazil, The Cambridge Blue pub serves delicious Tuscan Beef for the cheap price of £10.50. Delicious Pub food.
Role option scope: ['Team', 'Player']
Question: Dose the proper noun is a Role in the scope? If dose, which is?
Answer: Not Role

Below is the usr's question: 

Proper noun: {PRNON}
Related Context: {RELATED_CONTEXT}
Role option scope: {SCOP}
Answer:
"""

In [21]:
ie_prompt_extract_value_template="""
You are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.
1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.
2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.
3. Respond to the user's question like the examples provided below:

Role: Taste of Cambridge
Attribute: Price range
Related Context: Taste of Cambridge is a restaurant with a customer rating of 3 out of 5 and and a price range of £20-£25
Value scope: ['More than £30', 'Cheap', 'Less than £20', '£20-25', 'Moderate', 'High']
Question: What's the value of Taste of Cambridge's Price range?
Answer: £20-£25

Role: James Harden
Attribute: Points
Related Context: It was another day at work on Tuesday for superstar James Harden, who led the Rockets to a blowout victory with 34 points and 11 assists.
Value scope: integer
Question: What's the value of Player's Points?
Answer: 34

Role: Suns
Attribute: Number of team assists
Related Context: Memphis also registered 25 assists compared to only 13 - on 32 field goals - for the Suns
Value scope: integer
Question: What's the value of Suns's Number of team assists?
Answer: 13

Below is the usr's question:
    Role: {ROLE}
    Attribute: {FIELD}
    Related Context: {RELATED_CONTEXT}
    Value scope: {SCOPE}
    Question: What's the value of {ROLE}'s {FIELD}?
    Answer:
"""

In [22]:
ie_prompt_extract_role_name_e2e_template="""
You are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.
1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.
2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.
3. Respond to the user's question like the examples provided below:

Attribute: Restaurant Name
Related Context:  The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.
Question: What's is the Name of Restaurant
Answer: The Vaults pub

Attribute: Restaurant Name
Related Context:  Close to Café Brazil, The Cambridge Blue pub serves delicious Tuscan Beef for the cheap price of £10.50. Delicious Pub food.
Question: What's is the Name of Restaurant
Answer: The Cambridge Blue pub

Below is the usr's question:
    Role: {ROLE}
    Attribute: {FIELD}
    Related Context: {RELATED_CONTEXT}
    Question: What's the {FIELD} of {ROLE}?
    Answer:

"""

In [18]:
lines[1]

'Close to Café Brazil, The Cambridge Blue pub serves delicious Tuscan Beef for the cheap price of £10.50. Delicious Pub food.'

In [8]:
fields = fields_list_e2e['Table1']['Fields']
fields

{'Name': str,
 'Price range': ['More than £30',
  'Cheap',
  'Less than £20',
  '£20-25',
  'Moderate',
  'High'],
 'Customer rating': ['5 out of 5',
  'Low',
  'Average',
  '3 out of 5',
  '1 out of 5',
  'High'],
 'Near': str,
 'Food': ['Japanese',
  'French',
  'English',
  'Fast food',
  'Italian',
  'Indian',
  'Chinese'],
 'Area': ['Riverside', 'City centre'],
 'Family friendly': ['Yes', 'No']}

In [15]:
part = 'train'
table_data = pd.read_excel(f"../../data/e2e/{part}.xlsx")
table_data = table_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
table_data

/tmp/ipykernel_4185429/2306201433.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  table_data = table_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Unnamed: 0,Name,Price range,Customer rating,Near,Food,Area,Family friendly
0,0,The vaults,More than £30,5 out of 5,Café adriatic,NaN,NaN,NaN
1,1,The cambridge blue,Cheap,NaN,Café brazil,NaN,NaN,NaN
2,2,The eagle,Less than £20,Low,Burger king,Japanese,Riverside,Yes
3,3,The mill,£20-25,NaN,The sorrento,French,Riverside,NaN
4,4,Loch fyne,NaN,NaN,The rice boat,French,Riverside,NaN
...,...,...,...,...,...,...,...,...
42056,42056,The rice boat,Cheap,5 out of 5,Express by holiday inn,Indian,City centre,Yes
42057,42057,The vaults,NaN,NaN,NaN,Chinese,NaN,NaN
42058,42058,The cambridge blue,NaN,High,NaN,Italian,NaN,NaN
42059,42059,The eagle,Less than £20,NaN,Burger king,NaN,Riverside,Yes


In [16]:
text_corpus = lines[i]
print(f"Text Corpus: {text_corpus}")

corresponding_row = table_data.loc[i, :]
corresponding_row

Text Corpus: Located near The Sorrento is a French Theme eatery and coffee shop called The Mill, with a price range at £20-£25 it is in the riverside area.


Unnamed: 0                    3
Name                   The mill
Price range              £20-25
Customer rating             NaN
Near               The sorrento
Food                     French
Area                  Riverside
Family friendly             NaN
Name: 3, dtype: object

In [25]:
fields

{'Name': str,
 'Price range': ['More than £30',
  'Cheap',
  'Less than £20',
  '£20-25',
  'Moderate',
  'High'],
 'Customer rating': ['5 out of 5',
  'Low',
  'Average',
  '3 out of 5',
  '1 out of 5',
  'High'],
 'Near': str,
 'Food': ['Japanese',
  'French',
  'English',
  'Fast food',
  'Italian',
  'Indian',
  'Chinese'],
 'Area': ['Riverside', 'City centre'],
 'Family friendly': ['Yes', 'No']}

In [23]:
ft_data = []

In [28]:
res_role = "Restaurant"
field = "Name"
related_context = lines[i]
filled_template_role_hometeam_prompt = ie_prompt_extract_role_name_e2e_template.format(ROLE=res_role, FIELD=field, RELATED_CONTEXT=related_context)
filled_template_role_hometeam_prompt_answer = corresponding_row["Name"]
ft_data.append({"instruction": "Given the context, extract the restaurant name", "input": filled_template_role_hometeam_prompt, "output": filled_template_role_hometeam_prompt_answer})

for attribute, scope in fields.items():
    if attribute == "Name":
        continue
    role = corresponding_row["Name"]
    role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=attribute, SCOPE=scope, RELATED_CONTEXT=lines[i])
    role_value_answer = corresponding_row[attribute]
    if pd.isna(role_value_answer):
        role_value_answer = "No Info Found"
    ft_data.append({"instruction": "Given the context, extract the information", "input": role_value_prompt, "output": str(role_value_answer)})

In [30]:
ft_data = []
for i in range(len(lines)):
    res_role = "Restaurant"
    field = "Name"
    related_context = lines[i]
    corresponding_row = table_data.loc[i, :]
    filled_template_role_hometeam_prompt = ie_prompt_extract_role_name_e2e_template.format(ROLE=res_role, FIELD=field, RELATED_CONTEXT=related_context)
    filled_template_role_hometeam_prompt_answer = corresponding_row["Name"]
    ft_data.append({"instruction": "Given the context, extract the restaurant name", "input": filled_template_role_hometeam_prompt, "output": filled_template_role_hometeam_prompt_answer})

    for attribute, scope in fields.items():
        if attribute == "Name":
            continue
        role = corresponding_row["Name"]
        role_value_prompt = ie_prompt_extract_value_template.format(ROLE=role, FIELD=attribute, SCOPE=scope, RELATED_CONTEXT=lines[i])
        role_value_answer = corresponding_row[attribute]
        if pd.isna(role_value_answer):
            role_value_answer = "No Info Found"
        ft_data.append({"instruction": "Given the context, extract the information", "input": role_value_prompt, "output": str(role_value_answer)})

In [102]:
# Specify the filename
ft_data_path = "/data/zhaozibo/text-kgs-table/LLaMA-Factory/data"
filename = 'e2e_ie_train.json'
file_path = os.path.join(ft_data_path, filename)
# Open the file in write mode
with open(file_path, 'w') as json_file:
    json.dump(ft_data, json_file)

In [101]:
ft_data

[{'instruction': 'Given the context, extract the team name',
  'input': "\nYou are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.\n1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.\n2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.\n3. Respond to the user's question like the examples provided below:\n\nRole: Home team\nAttribute: Team Name\nRelated Context: The Los Angeles Clippers (3 - 1) defeated the Utah Jazz (1 - 3) 107 - 101 on Monday. Chris Paul recorded the NBA's first triple - double of the 2014 - 15 season, putting up 13 points, 10 rebounds, and 12 assists in 36 minutes. He also threw in two steals and a blocked shot for good measure. Teammate Bl

In [80]:
ft_data[:10]

[{'instruction': 'Given the context, extract the information',
  'input': "\nYou are a useful table content filling assistant that can extract the attribute field values of the role based on the row and column values provided by the user, as well as their corresponding paragraph original text, from the original text.\n1. Check if the provided paragraph contains the attribute values corresponding to the role. If not, respond 'Bad Infomation'.\n2. If the relevant paragraph contains attribute values for the role, respond the value according to the given requirements.\n3. Respond to the user's question like the examples provided below:\n\nRole: Home team\nAttribute: Team Name\nRelated Context: The Los Angeles Clippers (3 - 1) defeated the Utah Jazz (1 - 3) 107 - 101 on Monday. Chris Paul recorded the NBA's first triple - double of the 2014 - 15 season, putting up 13 points, 10 rebounds, and 12 assists in 36 minutes. He also threw in two steals and a blocked shot for good measure. Teammate 

In [43]:
dict(corresponding_table.loc[0, :])
ie_prompt_extract_role_name_rotowire_template.format()

{'Team Label': 'Home team ',
 'Team name': 'Clippers',
 'Number of team assists': nan,
 'Percentage of 3 points': nan,
 'Percentage of field goals': nan,
 'Losses': 1,
 'Total points': 107,
 'Points in 1st quarter': nan,
 'Points in 2nd quarter': nan,
 'Points in 3rd quarter': nan,
 'Points in 4th quarter': nan,
 'Rebounds': nan,
 'Turnovers': 10,
 'Wins': 3}

In [36]:
len(os.listdir(os.path.join("/data/zhaozibo/", "text-kgs-table/data/rotowire/tables/train"))), len(lines)

(3398, 3398)